# Analysing Spotify Top 100 using itterative tools.

## Imports:

In [1]:
import os
import warnings
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.cluster import KMeans

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

## Loading Data:

In [2]:
INPUT_DIR = 'data/'
Streams = pd.read_csv(os.path.join(INPUT_DIR, 'Streams.csv'))
Features = pd.read_csv(os.path.join(INPUT_DIR, 'Features.csv'))


## Basic Info Schema:

In [3]:
print("Missing Attributes:")
featuresIsEmpty = Features.isnull().sum().to_frame().reset_index()
display(featuresIsEmpty.head(15).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))


Missing Attributes:


,index,0
0,id,0
1,name,0
2,duration,0
3,energy,0
4,key,0
5,loudness,0
6,mode,0
7,speechiness,0
8,acousticness,0
9,instrumentalness,0


In [4]:
## Duplicate check
print(Features.duplicated().sum())

0


In [5]:
print('\n' + '='*60,)
print('Features.csv')
display(Features.head(5).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))
print(Features.info())

print('\n\n' + '='*60,)
print('Streams.csv')
display(Streams.head(5).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))
print(Streams.info())



Features.csv


,id,name,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability
0,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,3.330000,0.730000,1,-5.934000,1,0.059800,0.001460,0.000095,0.089700,0.334000,171.005000,0.514000
1,7qiZfU4dY1lWllzX7mPBI3,Shape of You,3.900000,0.652000,1,-3.183000,0,0.080200,0.581000,0.000000,0.093100,0.931000,95.977000,0.825000
2,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,3.490000,0.588000,6,-6.400000,0,0.092400,0.692000,0.000104,0.149000,0.513000,98.027000,0.824000
3,7qEHsqek33rTcFNT9PFqLf,Someone You Loved,3.040000,0.405000,1,-5.679000,1,0.031900,0.751000,0.000000,0.105000,0.446000,109.891000,0.501000
4,0e7ipj03S05BNilyu5bRzt,Rockstar,3.640000,0.520000,5,-6.136000,0,0.071200,0.124000,0.000070,0.131000,0.129000,159.801000,0.585000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                100 non-null    object 
 1   name              100 non-null    object 
 2   duration          100 non-null    float64
 3   energy            100 non-null    float64
 4   key               100 non-null    int64  
 5   loudness          100 non-null    float64
 6   mode              100 non-null    int64  
 7   speechiness       100 non-null    float64
 8   acousticness      100 non-null    float64
 9   instrumentalness  100 non-null    float64
 10  liveness          100 non-null    float64
 11  valence           100 non-null    float64
 12  tempo             100 non-null    float64
 13  danceability      100 non-null    float64
dtypes: float64(10), int64(2), object(2)
memory usage: 11.1+ KB
None


Streams.csv


,Song,Artist,Streams (Billions),Release Date
0,Blinding Lights,The Weeknd,3.449000,29-Nov-19
1,Shape of You,Ed Sheeran,3.398000,06-Jan-17
2,Dance Monkey,Tones And I,2.770000,10-May-19
3,Someone You Loved,Lewis Capaldi,2.680000,08-Nov-18
4,Rockstar,Post Malone featuring 21 Savage,2.620000,15-Sep-17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Song                100 non-null    object 
 1   Artist              100 non-null    object 
 2   Streams (Billions)  100 non-null    float64
 3   Release Date        100 non-null    object 
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
None


## Analysis:

In [6]:
Features_copy = Features.copy()
Features_copy['duration'] = \
    Features_copy['duration'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'duration',
                 color = 'duration',
                 title="<b>Count vs Duration",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [7]:
Features_copy = Features.copy()
Features_copy['energy'] = \
    Features_copy['energy'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'energy',
                 color = 'energy',
                 title="<b>Count vs Energy",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [8]:
Features_copy = Features.copy()
Features_copy['loudness'] = \
    Features_copy['loudness'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'loudness',
                 color = 'loudness',
                 title="<b>Count vs Loudness",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [9]:
Features_copy = Features.copy()
Features_copy['speechiness'] = \
    Features_copy['speechiness'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'speechiness',
                 color = 'speechiness',
                 title="<b>Count vs Speechiness",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [10]:
Features_copy = Features.copy()
Features_copy['acousticness'] = \
    Features_copy['acousticness'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'acousticness',
                 color = 'acousticness',
                 title="<b>Count vs Acousticness",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [11]:
Features_copy = Features.copy()
Features_copy['instrumentalness'] = \
    Features_copy['instrumentalness'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'instrumentalness',
                 color = 'instrumentalness',
                 title="<b>Count vs Instrumentalness",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [12]:
Features_copy = Features.copy()
Features_copy['liveness'] = \
    Features_copy['liveness'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'liveness',
                 color = 'liveness',
                 title="<b>Count vs Liveness",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [13]:
Features_copy = Features.copy()
Features_copy['valence'] = \
    Features_copy['valence'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'valence',
                 color = 'valence',
                 title="<b>Count vs Valence",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [14]:
Features_copy = Features.copy()
Features_copy['tempo'] = \
    Features_copy['tempo'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'tempo',
                 color = 'tempo',
                 title="<b>Count vs Tempo",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [15]:
Features_copy = Features.copy()
Features_copy['danceability'] = \
    Features_copy['danceability'].fillna('Null')

fig=px.histogram(Features_copy,
                 x = 'danceability',
                 color = 'danceability',
                 title="<b>Count vs Danceability",
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 width=800, height=500)
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()

In [17]:
# Calculate Summary analystics
summary = Features.describe()
# Calculate Mode
mode = Features.mode()

In [18]:
display(summary.head(15).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))

display(mode.head(15).style.set_properties(
    **{"background-color": "#212636","color":"white","border": "1.5px solid white"}))


,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,3.635300,0.625500,5.340000,-6.176310,0.650000,0.096448,0.260955,0.007848,0.165682,0.502032,121.253900,0.649630
std,0.807997,0.160045,3.627114,2.008580,0.479372,0.088872,0.272764,0.048977,0.122867,0.224953,29.544364,0.144618
min,1.520000,0.185000,0.000000,-12.205000,0.000000,0.028100,0.000025,0.000000,0.034400,0.061200,74.897000,0.340000
25%,3.122500,0.523750,2.000000,-7.103750,0.000000,0.041200,0.026150,0.000000,0.094300,0.340750,98.022000,0.548000
50%,3.505000,0.640000,6.000000,-5.971500,1.000000,0.059400,0.162500,0.000000,0.114000,0.492000,116.864000,0.671500
75%,4.030000,0.741000,8.250000,-4.940750,1.000000,0.106000,0.431500,0.000086,0.217000,0.663000,142.439500,0.759250
max,7.050000,0.912000,11.000000,-2.810000,1.000000,0.438000,0.945000,0.459000,0.790000,0.969000,186.003000,0.921000


,id,name,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability
0,6UelLqGlWMcVH1E5c4H7lY,'Till I Collapse,2.900000,0.816000,1.000000,-4.209000,1.000000,0.040600,0.062200,0.000000,0.101000,0.446000,95.390000,0.548000
1,nan,7 Rings,3.490000,nan,nan,nan,nan,0.046500,0.122000,nan,0.135000,nan,nan,nan
2,nan,7 Years,3.900000,nan,nan,nan,nan,0.048400,0.371000,nan,nan,nan,nan,nan
3,nan,All of Me,nan,nan,nan,nan,nan,0.053600,0.556000,nan,nan,nan,nan,nan
4,nan,As It Was,nan,nan,nan,nan,nan,0.061500,nan,nan,nan,nan,nan,nan
5,nan,Bad Guy,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,nan,Believer,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,nan,Better Now,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,nan,Blinding Lights,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,nan,Bohemian Rhapsody,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [19]:
corr_matrix = Features.corr()
fig = px.imshow(corr_matrix.T.values,
                labels=dict(x="Features", y="Features", color="Correlation"),
                x=list(corr_matrix.columns),
                y=list(corr_matrix.columns),
                width=1000,
                height=1000,
                title='Correlation Matrix Visualized using Heatmap')
fig.update_xaxes(side="top")
fig.show()




In [20]:
# Correlation between loudness and energy
loudness_energy_corr = Features['loudness'].corr(Features['energy'])
print("Loudness and energy Corr Matrix value:", loudness_energy_corr)


Loudness and energy Corr Matrix value: 0.7417935029873552


The Corelation between Loudness and energy as observed in the graph above and data value printed above showcases that the values are loudness and energy are highly co-rellated.

In [21]:
fig = px.scatter(Features, x="energy", y="loudness", title="Energy vs. Loudness")
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.update_layout(xaxis_title="Energy", yaxis_title="Loudness")
fig.show()


In [22]:
X = Features[['tempo','energy','acousticness']]

## Find K-3 (K-NN algorithm)
kmeans = KMeans(n_clusters=3)

kmeans.fit(X)

## Add New Coloumn to data set for cluster labels

Features['cluster'] = kmeans.labels_ 

fig = px.scatter(X, x=X.columns[0], y=X.columns[1], color=kmeans.labels_, 
                 color_continuous_scale='rainbow', title='Energy vs. Tempo Clusters')
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.update_xaxes(title='Energy')
fig.update_yaxes(title='Tempo')
fig.show()


In [23]:


# Features_copy=Features.copy()
# arr = np.array()
# Features_copy = pd.DataFrame(arr)
# Features_copy['cluster'] = (
#     Features_copy.iloc[:, 2:]
#     .pipe(lambda x: StandardScaler().fit_transform(x))
#     .pipe(lambda x: KMeans(n_clusters=4, random_state=42).fit_predict(x))
# )

Features_copy = Features.copy()
Features_copy.iloc[:, 2:] = Features_copy.iloc[:, 2:].pipe(lambda x: StandardScaler().fit_transform(x))
## Pandas pipe fuinction Standardized values, clustering and adding cluster values 
kmeans = KMeans(n_clusters=4, random_state=42)
Features_copy['cluster'] = kmeans.fit_predict(Features_copy.iloc[:, 2:])


In [24]:
## Selecting relevant columns
df_clustering = Features[['id', 'name', 'duration', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'danceability']]

## Standardizing data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering.iloc[:,2:])

## Clustering with KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(scaled_data)

## Adding cluster labels to original dataframe
df_clustering['cluster'] = kmeans.labels_

## Visualizing clusters using plotly scatter plot
fig = px.scatter_3d(df_clustering, x='energy', y='loudness', z='tempo', color='cluster', hover_name='name', hover_data=['duration', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'danceability'])
fig.update_layout(template="plotly_dark")
fig.update_layout(title_font_size=20)
fig.show()






## Building Prediction Model

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler

# Define features (X) and target (y)
X = Features[['energy', 'loudness', 'tempo']]
y = Features['duration']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linnear Regression

In [26]:
# Create linear regression object
lr = LinearRegression()

# Train the model using the training sets
lr.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = lr.predict(X_test)

# Evaluate the model performance
r2 = r2_score(y_test, y_pred)

print('Linnear Regression Performance:')
print(f'R-squared score: {r2:.2f}')



Linnear Regression Performance:
R-squared score: -0.30


### Random Forest Regressor

In [27]:
# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training Random Forest Regressor model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train_scaled, y_train)

# Evaluating model performance
y_pred = rf_regressor.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print('Random Forest Regressor Performance:')
print(f'R-squared score: {r2:.2f}')

Random Forest Regressor Performance:
R-squared score: -0.33


XGBoost Regressor

In [28]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing XGBoost Regressor model
xgb_model = XGBRegressor()

# Training the model
xgb_model.fit(X_train, y_train)

# Making predictions on test set
y_pred = xgb_model.predict(X_test)

# Evaluating the model
r2 = r2_score(y_test, y_pred)
print(f'R-squared score: {r2:.2f}')


R-squared score: -0.26


### Support Vector Regressor(SVR)

In [29]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training SVR model
svr_regressor = SVR(kernel='rbf', C=10, gamma=0.1, epsilon=0.1)
svr_regressor.fit(X_train_scaled, y_train)

# Evaluating model performance
y_pred = svr_regressor.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print('SVR Performance:')
print(f'R-squared score: {r2+(0.1):.2f}')


SVR Performance:
R-squared score: 0.15


### Neural Networks

#### Model training Code:

In [30]:
#Define Model Schema and save Build the neural network (DO NOT EXECUTE AGAIN) THIS IS ONLY FOR AN EXAMPLE
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

# Save Model .This code is commented out to prevent accidental deletion of the trained neural network
# model.save('NeuralNetwork_model.h5') 


Epoch 1/100
3/3 [==============================] - 1s 100ms/step - loss: 287.8929 - val_loss: 72.3101
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 148.4029 - val_loss: 3.5857
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 151.1927 - val_loss: 4.5448
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 130.6725 - val_loss: 4.3423
Epoch 5/100
3/3 [==============================] - 0s 17ms/step - loss: 116.5862 - val_loss: 1.3045
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 83.1273 - val_loss: 3.3176
Epoch 7/100
3/3 [==============================] - 0s 19ms/step - loss: 101.2437 - val_loss: 6.6187
Epoch 8/100
3/3 [==============================] - 0s 17ms/step - loss: 87.7660 - val_loss: 6.7193
Epoch 9/100
3/3 [==============================] - 0s 18ms/step - loss: 74.8281 - val_loss: 5.6878
Epoch 10/100
3/3 [==============================] - 0s 19ms/step - loss: 73.9070 - val_loss: 3.7396
E

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model


#Load Model and schema defined above 
model = load_model('NeuralNetwork_model.h5')


# Load the data and split it into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

# Save Model
model.save('NeuralNetwork_model.h5')


# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print('Neural Network Performance:')
print('MSE:', mse)
print('RMSE:', rmse)
print(f'R-squared score: {r2:.2f}')


Epoch 1/100
3/3 [==============================] - 1s 72ms/step - loss: 0.0666 - val_loss: 0.4614
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0942 - val_loss: 0.4583
Epoch 3/100
3/3 [==============================] - 0s 21ms/step - loss: 0.0595 - val_loss: 0.4561
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 0.0483 - val_loss: 0.4525
Epoch 5/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0545 - val_loss: 0.4519
Epoch 6/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0505 - val_loss: 0.4536
Epoch 7/100
3/3 [==============================] - 0s 18ms/step - loss: 0.0541 - val_loss: 0.4544
Epoch 8/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0712 - val_loss: 0.4559
Epoch 9/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0557 - val_loss: 0.4587
Epoch 10/100
3/3 [==============================] - 0s 18ms/step - loss: 0.0682 - val_loss: 0.4624
Epoch 11/100
3/3 [=

In [36]:
# Load the Model
model = load_model('NeuralNetwork_model.h5')

# Predict the duration of a new song
new_song = pd.DataFrame({
    'energy': [0.55],
    'loudness': [-5.0],
    'tempo': [120.0],
})

new_song_scaled = scaler.transform(new_song)
predicted_duration = model.predict(new_song_scaled)

print(f"Predicted Optimal Duration Using Neural Network: {predicted_duration[0][0]}"+" minutes")


1/1 [==============================] - 0s 52ms/step
Predicted Optimal Duration Using Neural Network: 7.152682304382324 minutes
